In [29]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [30]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
file_path = Path('Resources/lending_data.csv')
lending_data_df = pd.read_csv(file_path)

# Review the DataFrame
lending_data_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


In [31]:
# get the shape of the dataframe
lending_data_df.shape

(77536, 8)

### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [32]:
# Separate the data into labels and features
# Separate the y variable, the labels
y = lending_data_df["loan_status"].values.reshape(-1, 1).ravel()


# Separate the X variable, the features
X = lending_data_df.copy()
X.drop("loan_status", axis=1, inplace=True)

In [33]:
# Review the y variable Series
y[:5]

array([0, 0, 0, 0, 0])

In [34]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [35]:
# Check the balance of our target values
loan_status_balance = lending_data_df["loan_status"].value_counts()

print(loan_status_balance)

0    75036
1     2500
Name: loan_status, dtype: int64


### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [36]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [37]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using training data
logistic_classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [38]:
# Make a prediction using the testing data
logistic_predictions = logistic_classifier.predict(X_test)

logistic_pred_df = pd.DataFrame({"Prediction": logistic_predictions, "Actual": y_test}).reset_index(drop = True)
logistic_pred_df.head(5)

,Prediction,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [39]:
logistic_pred_df.tail(5)

,Prediction,Actual
19379,0,0
19380,0,0
19381,0,0
19382,0,0
19383,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [40]:
# Print the balanced_accuracy score of the model
print("balanced accuracy score: %.3f" % balanced_accuracy_score(y_test, logistic_predictions))

balanced accuracy score: 0.952


In [41]:
# Generate a confusion matrix for the model
conf_matrix = confusion_matrix(y_test, logistic_predictions)

In [42]:
# Generate the classification report for the model
classification_rep = classification_report(y_test, logistic_predictions)

# Print the classification report
print("Classification Report:")
print(classification_rep)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The model does an excellent job at telling us which loans are "healthy" (0) with very high accuracy. It's not as perfect at identifying "high-risk" loans (1), but it still does a pretty good job. In general, the model works well for both, but it's especially good at spotting "healthy loans."

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [43]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
random_oversampler = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_train_resampled, y_train_resampled = random_oversampler.fit_resample(X_train, y_train)

In [44]:
# Count the distinct values in the resampled labels data (y_train_resampled)
distinct_values = np.unique(y_train_resampled)

# Print the distinct values and their counts
for value in distinct_values:
    count = np.sum(y_train_resampled == value)
    print(f"Label {value}: Count {count}")

Label 0: Count 56271
Label 1: Count 56271


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [45]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_classifier_resampled = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
logistic_classifier_resampled.fit(X_train_resampled, y_train_resampled)

# Make a prediction using the testing data
logistic_predictions_resampled = logistic_classifier_resampled.predict(X_test)


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [46]:
# Calculate the balanced accuracy score for the model
balanced_accuracy = balanced_accuracy_score(y_test, logistic_predictions_resampled)

# Print the balanced accuracy score
print("Balanced Accuracy Score:", balanced_accuracy)


Balanced Accuracy Score: 0.9936781215845847


In [47]:
# Generate a confusion matrix for the model
conf_matrix_resampled = confusion_matrix(y_test, logistic_predictions_resampled)

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_matrix_resampled)


Confusion Matrix:
[[18649   116]
 [    4   615]]


In [48]:
# Generate the classification report for the model
classification_rep_resampled = classification_report(y_test, logistic_predictions_resampled)

# Print the classification report
print("Classification Report:")
print(classification_rep_resampled)


Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.84      0.99      0.91       619

    accuracy                           0.99     19384
   macro avg       0.92      0.99      0.95     19384
weighted avg       0.99      0.99      0.99     19384

